# 🌊 해상 Shot Gather 대화형 워크플로우
# Marine Shot Gather Interactive Workflow

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/knocgp/seismic/blob/main/Shot_Gather_Interactive.ipynb)

---

## 📋 단계별 실행 워크플로우

**각 셀을 하나씩 실행하면서 결과를 확인하세요!**

1. ✅ 패키지 설치 및 클래스 정의
2. ✅ 랜덤 합성 모델 생성 → 즉시 시각화
3. ✅ Shot Gather 생성 (Clean) → 즉시 시각화
4. ✅ 직접파 추가 → 즉시 시각화
5. ✅ Multiple 추가 (해면 + 내부) → 즉시 시각화
6. ✅ 해상 노이즈 추가 → 즉시 시각화
7. ✅ Multiple 제거 → 즉시 시각화
8. ✅ 노이즈 제거 → 즉시 시각화
9. ✅ 전체 비교 → 즉시 시각화
10. ✅ 데이터 저장 및 다운로드
11. ✅ 추가 분석

---

**🚀 사용법: 각 셀을 순서대로 실행 (Shift + Enter)**

## 📦 Step 1: 패키지 설치 및 임포트

In [ ]:
!pip install -q numpy scipy matplotlib

import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from scipy.ndimage import median_filter
from typing import Tuple, Dict
import warnings
warnings.filterwarnings('ignore')

print("✅ 패키지 설치 및 임포트 완료!")
print("   - NumPy")
print("   - SciPy")
print("   - Matplotlib")

## 🔧 Step 1-2: MarineShotGatherProcessor 클래스 정의

**해상 탄성파 특화:**
- ✅ 직접파 (Direct Wave)
- ✅ 해면 Multiple (Sea Surface Multiple)
- ✅ 내부 Multiple (Internal Multiple)
- ✅ 해상 노이즈 (선박, 스웰, 버스트)
- ❌ Ground Roll 없음 (해상 특성)

In [ ]:
class MarineShotGatherProcessor:
    """해상 Shot Gather 생성 및 처리 클래스"""
    
    def __init__(self, dt: float = 0.002, nt: int = 1500):
        self.dt = dt
        self.nt = nt
        self.time = np.arange(nt) * dt
        
    def create_random_model(self, nlayers: int = None) -> Dict:
        """완전 랜덤 합성 지반 모델 생성"""
        if nlayers is None:
            nlayers = np.random.randint(4, 9)
        
        model = {'velocity': [], 'density': [], 'thickness': [], 'depth': [], 'name': []}
        
        # 해수층
        water_depth = np.random.uniform(300, 800)
        model['velocity'].append(1500.0)
        model['density'].append(1030.0)
        model['thickness'].append(water_depth)
        model['depth'].append(0.0)
        model['name'].append('Water')
        
        # 해저면
        seabed_vp = np.random.uniform(1600, 2000)
        seabed_rho = np.random.uniform(1900, 2100)
        seabed_thick = np.random.uniform(200, 400)
        model['velocity'].append(seabed_vp)
        model['density'].append(seabed_rho)
        model['thickness'].append(seabed_thick)
        model['depth'].append(water_depth)
        model['name'].append('Seabed')
        
        # 지하 지층들
        current_depth = water_depth + seabed_thick
        for i in range(nlayers - 2):
            if i == 0:
                base_vp = seabed_vp + np.random.uniform(200, 500)
            else:
                base_vp = model['velocity'][-1] + np.random.uniform(100, 600)
            
            vp = base_vp + np.random.normal(0, 100)
            vp = np.clip(vp, 2000, 5000)
            rho = 2000 + (vp - 2000) * 0.2 + np.random.normal(0, 50)
            rho = np.clip(rho, 2000, 2800)
            thickness = np.random.uniform(150, 600)
            
            model['velocity'].append(vp)
            model['density'].append(rho)
            model['thickness'].append(thickness)
            model['depth'].append(current_depth)
            model['name'].append(f'Layer {i+3}')
            current_depth += thickness
        
        return model
    
    def calculate_reflection_coefficients(self, model: Dict):
        velocities = np.array(model['velocity'])
        densities = np.array(model['density'])
        thicknesses = np.array(model['thickness'])
        impedance = velocities * densities
        
        rc = np.zeros(len(velocities) - 1)
        for i in range(len(velocities) - 1):
            rc[i] = (impedance[i+1] - impedance[i]) / (impedance[i+1] + impedance[i])
        
        times = np.zeros(len(velocities) - 1)
        cumulative_time = 0
        for i in range(len(velocities) - 1):
            travel_time = thicknesses[i] / velocities[i]
            cumulative_time += travel_time
            times[i] = cumulative_time * 2
        
        return rc, times
    
    def ricker_wavelet(self, freq: float = 25.0):
        duration = 0.2
        t = np.arange(-duration/2, duration/2, self.dt)
        a = (np.pi * freq * t) ** 2
        wavelet = (1 - 2*a) * np.exp(-a)
        return wavelet / np.max(np.abs(wavelet))
    
    def generate_shot_gather(self, model: Dict, n_traces: int = 48, 
                           offset_min: float = 100, offset_max: float = 2400,
                           freq: float = 25.0):
        """Shot Gather 생성 (반사파만)"""
        offsets = np.linspace(offset_min, offset_max, n_traces)
        shot_gather = np.zeros((self.nt, n_traces))
        wavelet = self.ricker_wavelet(freq)
        rc, zero_offset_times = self.calculate_reflection_coefficients(model)
        
        for i_trace, offset in enumerate(offsets):
            reflectivity = np.zeros(self.nt)
            
            for j, (rc_val, t0) in enumerate(zip(rc, zero_offset_times)):
                depths = np.array(model['depth'])
                velocities = np.array(model['velocity'])
                
                if j < len(depths) - 1:
                    avg_depth = depths[j+1]
                    avg_velocity = np.mean(velocities[:j+2])
                    t_nmo = np.sqrt(t0**2 + (offset / avg_velocity)**2)
                    angle = np.arctan(offset / avg_depth)
                    avo_factor = 1 - 0.3 * np.sin(angle)**2
                    
                    idx = int(t_nmo / self.dt)
                    if idx < self.nt:
                        reflectivity[idx] += rc_val * avo_factor
            
            trace = signal.convolve(reflectivity, wavelet, mode='same')
            spreading = 1 / (1 + offset / 1000)
            shot_gather[:, i_trace] = trace * spreading
        
        return shot_gather, offsets
    
    def add_direct_wave(self, shot_gather, offsets, model: Dict, strength: float = 0.3):
        """직접파 추가 (해수층 통과)"""
        result = shot_gather.copy()
        water_velocity = model['velocity'][0]
        wavelet = self.ricker_wavelet(25.0)
        
        for i, offset in enumerate(offsets):
            # 직접파 도달 시간
            direct_time = offset / water_velocity
            idx = int(direct_time / self.dt)
            
            if idx < self.nt:
                # 거리에 따른 감쇠
                amplitude = strength / (1 + offset / 500)
                
                # Wavelet 추가
                wavelet_start = max(0, idx - len(wavelet)//2)
                wavelet_end = min(self.nt, idx + len(wavelet)//2)
                wavelet_idx_start = max(0, len(wavelet)//2 - idx)
                wavelet_idx_end = wavelet_idx_start + (wavelet_end - wavelet_start)
                
                result[wavelet_start:wavelet_end, i] += amplitude * wavelet[wavelet_idx_start:wavelet_idx_end]
        
        return result
    
    def add_sea_surface_multiple(self, shot_gather, model: Dict, strength: float = 0.5):
        """해면 멀티플 추가"""
        result = shot_gather.copy()
        water_depth = model['thickness'][0]
        water_velocity = model['velocity'][0]
        two_way_time = 2 * water_depth / water_velocity
        delay_samples = int(two_way_time / self.dt)
        sea_surface_rc = -0.95
        
        # 1차 멀티플
        if delay_samples < self.nt:
            result[delay_samples:, :] += shot_gather[:-delay_samples, :] * sea_surface_rc * strength
        
        # 2차 멀티플
        if 2 * delay_samples < self.nt:
            result[2*delay_samples:, :] += shot_gather[:-2*delay_samples, :] * (sea_surface_rc**2) * strength * 0.5
        
        return result
    
    def add_internal_multiples(self, shot_gather, model: Dict, strength: float = 0.3):
        """내부 멀티플 추가"""
        result = shot_gather.copy()
        rc, reflection_times = self.calculate_reflection_coefficients(model)
        
        strong_reflectors = [(t, rc_val) for t, rc_val in zip(reflection_times, rc) 
                           if abs(rc_val) > 0.1]
        
        for i, (t1, rc1) in enumerate(strong_reflectors):
            for t2, rc2 in strong_reflectors[i+1:]:
                multiple_delay = t2 - t1 + (t2 - t1)
                delay_samples = int(multiple_delay / self.dt)
                
                if delay_samples < self.nt:
                    multiple_strength = rc1 * rc2 * strength
                    result[delay_samples:, :] += shot_gather[:-delay_samples, :] * multiple_strength
        
        return result
    
    def add_marine_noise(self, shot_gather, noise_level: float = 0.08):
        """해상 노이즈 추가 (Ground Roll 제외)"""
        result = shot_gather.copy()
        signal_power = np.std(shot_gather)
        nt, n_traces = shot_gather.shape
        
        # 1. 백색 잡음
        white_noise = np.random.normal(0, noise_level * signal_power * 0.3, (nt, n_traces))
        result += white_noise
        
        # 2. 선박 노이즈 (2-8 Hz)
        ship_freq = np.random.uniform(2, 8)
        for j in range(n_traces):
            ship_noise = noise_level * signal_power * 0.5 * np.sin(2 * np.pi * ship_freq * self.time)
            ship_noise *= (1 + 0.3 * np.sin(2 * np.pi * 0.5 * self.time))
            result[:, j] += ship_noise
        
        # 3. 스웰 노이즈 (0.1-0.5 Hz)
        swell_freq = np.random.uniform(0.1, 0.5)
        for j in range(n_traces):
            swell_noise = noise_level * signal_power * 0.4 * np.sin(2 * np.pi * swell_freq * self.time)
            swell_noise *= (1 + 0.5 * np.sin(2 * np.pi * 0.2 * self.time))
            result[:, j] += swell_noise
        
        # 4. 버스트 노이즈 (간헐적)
        n_bursts = np.random.randint(2, 5)
        for _ in range(n_bursts):
            burst_trace = np.random.randint(0, n_traces)
            burst_time = np.random.randint(0, nt)
            burst_duration = np.random.randint(20, 80)
            if burst_time + burst_duration < nt:
                burst = noise_level * signal_power * 2.0 * np.random.randn(burst_duration)
                result[burst_time:burst_time+burst_duration, burst_trace] += burst
        
        return result
    
    def demultiple_radon(self, shot_gather, strength: float = 0.7):
        """Radon 변환 기반 멀티플 제거 (간단 버전)"""
        # 간단한 적응 필터링
        result = shot_gather.copy()
        
        # 시간 방향 median 필터 (멀티플 억제)
        for i in range(shot_gather.shape[1]):
            trace = shot_gather[:, i]
            filtered = median_filter(trace, size=15)
            result[:, i] = trace - strength * (filtered - trace)
        
        return result
    
    def denoise_bandpass_filter(self, shot_gather, low_freq: float = 8.0, high_freq: float = 60.0):
        """밴드패스 필터"""
        nt, n_traces = shot_gather.shape
        denoised = np.zeros_like(shot_gather)
        nyquist = 1 / (2 * self.dt)
        low = low_freq / nyquist
        high = high_freq / nyquist
        b, a = signal.butter(4, [low, high], btype='band')
        
        for i in range(n_traces):
            denoised[:, i] = signal.filtfilt(b, a, shot_gather[:, i])
        return denoised
    
    def denoise_median_filter(self, shot_gather, size: int = 5):
        """Median 필터"""
        return median_filter(shot_gather, size=(size, 1))
    
    def denoise_fk_filter(self, shot_gather, velocity_cutoff: float = 1500):
        """F-K 필터"""
        nt, n_traces = shot_gather.shape
        fk_spectrum = np.fft.fft2(shot_gather)
        fk_spectrum_shifted = np.fft.fftshift(fk_spectrum)
        
        freq = np.fft.fftshift(np.fft.fftfreq(nt, self.dt))
        k = np.fft.fftshift(np.fft.fftfreq(n_traces, 50))
        
        fk_filter = np.ones_like(fk_spectrum_shifted)
        for i, f in enumerate(freq):
            for j, kval in enumerate(k):
                if f != 0 and kval != 0:
                    apparent_velocity = abs(f / kval)
                    if apparent_velocity < velocity_cutoff:
                        fk_filter[i, j] = 0.1
        
        fk_filtered = fk_spectrum_shifted * fk_filter
        fk_filtered_unshifted = np.fft.ifftshift(fk_filtered)
        return np.real(np.fft.ifft2(fk_filtered_unshifted))
    
    def plot_model(self, model: Dict):
        """지층 모델 시각화"""
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 8))
        
        depths = model['depth']
        velocities = model['velocity']
        densities = model['density']
        
        for i in range(len(depths)):
            depth_top = depths[i]
            depth_bottom = depths[i] + model['thickness'][i]
            
            ax1.fill_between([velocities[i]-100, velocities[i]+100],
                            depth_top, depth_bottom,
                            alpha=0.4, label=model['name'][i] if i < 5 else None)
            ax1.plot([velocities[i], velocities[i]], [depth_top, depth_bottom],
                    'b-', linewidth=2.5)
            
            ax2.fill_between([densities[i]-50, densities[i]+50],
                            depth_top, depth_bottom,
                            alpha=0.4)
            ax2.plot([densities[i], densities[i]], [depth_top, depth_bottom],
                    'r-', linewidth=2.5)
        
        ax1.set_xlabel('Velocity (m/s)', fontsize=13, fontweight='bold')
        ax1.set_ylabel('Depth (m)', fontsize=13, fontweight='bold')
        ax1.set_title('Velocity Model', fontsize=15, fontweight='bold')
        ax1.invert_yaxis()
        ax1.grid(True, alpha=0.4)
        ax1.legend(fontsize=10)
        
        ax2.set_xlabel('Density (kg/m³)', fontsize=13, fontweight='bold')
        ax2.set_ylabel('Depth (m)', fontsize=13, fontweight='bold')
        ax2.set_title('Density Model', fontsize=15, fontweight='bold')
        ax2.invert_yaxis()
        ax2.grid(True, alpha=0.4)
        
        plt.tight_layout()
        plt.show()
    
    def plot_shot_gather(self, shot_gather, offsets, title: str = "Shot Gather", clip_percentile: float = 99):
        """Shot Gather 시각화"""
        fig, ax = plt.subplots(figsize=(14, 10))
        vmax = np.percentile(np.abs(shot_gather), clip_percentile)
        
        for i, offset in enumerate(offsets):
            trace = shot_gather[:, i]
            trace_scaled = trace / vmax * 30
            ax.plot(offset + trace_scaled, self.time, 'k-', linewidth=0.3)
            ax.fill_betweenx(self.time, offset, offset + trace_scaled,
                            where=(trace_scaled > 0), color='black', alpha=0.6)
        
        ax.set_xlabel('Offset (m)', fontsize=13, fontweight='bold')
        ax.set_ylabel('Time (s)', fontsize=13, fontweight='bold')
        ax.set_title(title, fontsize=15, fontweight='bold')
        ax.invert_yaxis()
        ax.grid(True, alpha=0.3, linestyle='--')
        ax.set_xlim([offsets[0] - 100, offsets[-1] + 100])
        plt.tight_layout()
        plt.show()
    
    def plot_comparison_4(self, data1, data2, data3, data4, offsets, titles):
        """4개 비교"""
        fig, axes = plt.subplots(1, 4, figsize=(24, 10))
        data_list = [data1, data2, data3, data4]
        vmax = np.percentile(np.abs(data1), 99)
        
        for ax, data, title in zip(axes, data_list, titles):
            for i, offset in enumerate(offsets):
                trace = data[:, i]
                trace_scaled = trace / vmax * 30
                ax.plot(offset + trace_scaled, self.time, 'k-', linewidth=0.3)
                ax.fill_betweenx(self.time, offset, offset + trace_scaled,
                                where=(trace_scaled > 0), color='black', alpha=0.6)
            
            ax.set_xlabel('Offset (m)', fontsize=11, fontweight='bold')
            ax.set_ylabel('Time (s)', fontsize=11, fontweight='bold')
            ax.set_title(title, fontsize=13, fontweight='bold')
            ax.invert_yaxis()
            ax.grid(True, alpha=0.3, linestyle='--')
            ax.set_xlim([offsets[0] - 100, offsets[-1] + 100])
        
        plt.tight_layout()
        plt.show()

print("✅ MarineShotGatherProcessor 클래스 정의 완료!")
print("\n해상 탄성파 특화 기능:")
print("  ✅ 직접파 (Direct Wave)")
print("  ✅ 해면 Multiple (Sea Surface)")
print("  ✅ 내부 Multiple (Internal)")
print("  ✅ 해상 노이즈 (선박, 스웰, 버스트)")
print("  ❌ Ground Roll 제외 (해상)")

## 🌍 Step 2: 랜덤 합성 모델 생성

**이 셀을 실행하면:**
- 완전 랜덤 지층 모델 생성 (해수층 포함)
- 지층 정보 테이블 출력
- 속도 및 밀도 프로파일 시각화

In [ ]:
# 프로세서 초기화
processor = MarineShotGatherProcessor(dt=0.002, nt=1500)
print("✅ 프로세서 초기화 완료")
print(f"   - 샘플링 간격: {processor.dt*1000:.1f} ms")
print(f"   - 시간 샘플: {processor.nt}개")
print(f"   - 총 시간: {processor.time[-1]:.2f} s")
print()

# 랜덤 모델 생성
print("🌍 랜덤 합성 지반 모델 생성 중...")
model = processor.create_random_model(nlayers=6)
print("✅ 모델 생성 완료!")
print()

# 지층 정보 출력
print("="*80)
print("📊 생성된 지층 정보")
print("="*80)
print(f"{'Layer':<15} {'Depth (m)':<12} {'Thickness (m)':<15} {'Velocity (m/s)':<15} {'Density (kg/m³)'}")
print("-"*80)
for i in range(len(model['name'])):
    print(f"{model['name'][i]:<15} {model['depth'][i]:<12.1f} {model['thickness'][i]:<15.1f} "
          f"{model['velocity'][i]:<15.1f} {model['density'][i]:<15.1f}")
print("="*80)
print()

# 모델 시각화
print("📈 지층 모델 시각화...")
processor.plot_model(model)

## 🎯 Step 3: Shot Gather 생성 (Clean - 반사파만)

**이 셀을 실행하면:**
- 48개 트레이스 Shot Gather 생성
- NMO 및 AVO 효과 적용
- 반사파만 포함 (직접파, Multiple, 노이즈 없음)
- 원본 Shot Gather 시각화

In [ ]:
print("🎯 Clean Shot Gather 생성 중 (반사파만)...")
print()

# Shot Gather 생성
n_traces = 48
offset_min = 100
offset_max = 2400
freq = 25.0

clean_shot, offsets = processor.generate_shot_gather(
    model,
    n_traces=n_traces,
    offset_min=offset_min,
    offset_max=offset_max,
    freq=freq
)

print("✅ Clean Shot Gather 생성 완료!")
print()
print("📊 Shot Gather 정보:")
print(f"   - 트레이스 개수: {n_traces}")
print(f"   - 오프셋 범위: {offsets[0]:.0f} ~ {offsets[-1]:.0f} m")
print(f"   - Wavelet 주파수: {freq} Hz")
print(f"   - 데이터 크기: {clean_shot.shape}")
print(f"   - RMS: {np.sqrt(np.mean(clean_shot**2)):.6f}")
print()

# 시각화
print("📈 Clean Shot Gather 시각화 (반사파만)...")
processor.plot_shot_gather(clean_shot, offsets, "✨ Clean Shot Gather (Reflections Only)")

## 🌊 Step 4: 직접파 추가 (Direct Wave)

**이 셀을 실행하면:**
- 해수층을 통과하는 직접파 추가
- 오프셋에 따른 도달 시간 계산
- 거리에 따른 감쇠 적용
- 직접파가 추가된 Shot Gather 시각화

In [ ]:
print("🌊 직접파 추가 중...")
print()

# 직접파 추가
direct_strength = 0.3
with_direct = processor.add_direct_wave(clean_shot, offsets, model, strength=direct_strength)

print("✅ 직접파 추가 완료!")
print()
print("📊 직접파 정보:")
print(f"   - 전파 속도: {model['velocity'][0]:.0f} m/s (해수층)")
print(f"   - 강도: {direct_strength}")
print(f"   - 최소 도달 시간: {offsets[0]/model['velocity'][0]:.3f} s")
print(f"   - 최대 도달 시간: {offsets[-1]/model['velocity'][0]:.3f} s")
print()

# 통계
direct_only = with_direct - clean_shot
print("📈 통계:")
print(f"   - Clean RMS: {np.sqrt(np.mean(clean_shot**2)):.6f}")
print(f"   - With Direct RMS: {np.sqrt(np.mean(with_direct**2)):.6f}")
print(f"   - Direct Wave RMS: {np.sqrt(np.mean(direct_only**2)):.6f}")
print()

# 시각화
print("📈 직접파가 추가된 Shot Gather 시각화...")
processor.plot_shot_gather(with_direct, offsets, "🌊 Shot Gather with Direct Wave")

## 🔄 Step 5: Multiple 추가 (해면 + 내부)

**이 셀을 실행하면:**
- 해면 Multiple 추가 (1차, 2차)
- 내부 Multiple 추가 (강한 반사면 간)
- Multiple이 추가된 Shot Gather 시각화

In [ ]:
print("🔄 Multiple 추가 중...")
print()

# 해면 Multiple 추가
print("1️⃣ 해면 Multiple 추가...")
sea_mult_strength = 0.5
with_sea_mult = processor.add_sea_surface_multiple(with_direct, model, strength=sea_mult_strength)
print(f"   ✅ 해면 Multiple 강도: {sea_mult_strength}")
print(f"   ✅ 해수층 양방향 주시: {2*model['thickness'][0]/model['velocity'][0]:.3f} s")
print()

# 내부 Multiple 추가
print("2️⃣ 내부 Multiple 추가...")
int_mult_strength = 0.3
with_multiples = processor.add_internal_multiples(with_sea_mult, model, strength=int_mult_strength)
print(f"   ✅ 내부 Multiple 강도: {int_mult_strength}")
print()

print("✅ Multiple 추가 완료!")
print()

# 통계
multiples_only = with_multiples - with_direct
print("📈 통계:")
print(f"   - Before Multiples RMS: {np.sqrt(np.mean(with_direct**2)):.6f}")
print(f"   - With Multiples RMS: {np.sqrt(np.mean(with_multiples**2)):.6f}")
print(f"   - Multiples Only RMS: {np.sqrt(np.mean(multiples_only**2)):.6f}")
print()

# 시각화
print("📈 Multiple이 추가된 Shot Gather 시각화...")
processor.plot_shot_gather(with_multiples, offsets, "🔄 Shot Gather with Multiples")

## 📢 Step 6: 해상 노이즈 추가

**이 셀을 실행하면:**
- 백색 잡음
- 선박 노이즈 (2-8 Hz)
- 스웰 노이즈 (0.1-0.5 Hz)
- 버스트 노이즈 (간헐적)
- ❌ Ground Roll 없음 (해상 특성)
- 노이즈가 추가된 Shot Gather 시각화

In [ ]:
print("📢 해상 노이즈 추가 중...")
print()

# 노이즈 추가
noise_level = 0.08
noisy_shot = processor.add_marine_noise(with_multiples, noise_level=noise_level)

print("✅ 해상 노이즈 추가 완료!")
print()
print("📊 추가된 노이즈:")
print("   ✅ 백색 잡음 (White Noise)")
print("   ✅ 선박 노이즈 (Ship Noise, 2-8 Hz)")
print("   ✅ 스웰 노이즈 (Swell Noise, 0.1-0.5 Hz)")
print("   ✅ 버스트 노이즈 (Burst Noise)")
print("   ❌ Ground Roll 없음 (해상 환경)")
print()

# 통계
noise = noisy_shot - with_multiples
snr_before = 20 * np.log10(np.std(with_multiples) / np.std(noise))

print("📈 통계:")
print(f"   - Signal RMS: {np.sqrt(np.mean(with_multiples**2)):.6f}")
print(f"   - Noisy RMS: {np.sqrt(np.mean(noisy_shot**2)):.6f}")
print(f"   - Noise RMS: {np.sqrt(np.mean(noise**2)):.6f}")
print(f"   - SNR: {snr_before:.2f} dB")
print()

# 시각화
print("📈 노이즈가 추가된 Shot Gather 시각화...")
processor.plot_shot_gather(noisy_shot, offsets, "📢 Shot Gather with Marine Noise")

## 🧹 Step 7: Multiple 제거 (Demultiple)

**이 셀을 실행하면:**
- Radon 변환 기반 Multiple 억제
- 적응 필터링
- Multiple 제거된 Shot Gather 시각화

In [ ]:
print("🧹 Multiple 제거 중...")
print()
print("적용 기법:")
print("   - Radon 변환 기반 Multiple 억제")
print("   - 적응 필터링")
print()

# Multiple 제거
demult_strength = 0.7
demultipled = processor.demultiple_radon(noisy_shot, strength=demult_strength)

print("✅ Multiple 제거 완료!")
print()

# 통계
removed_mult = noisy_shot - demultipled

print("📈 통계:")
print(f"   - Before Demultiple RMS: {np.sqrt(np.mean(noisy_shot**2)):.6f}")
print(f"   - After Demultiple RMS: {np.sqrt(np.mean(demultipled**2)):.6f}")
print(f"   - Removed Multiples RMS: {np.sqrt(np.mean(removed_mult**2)):.6f}")
print()

# 시각화
print("📈 Multiple 제거된 Shot Gather 시각화...")
processor.plot_shot_gather(demultipled, offsets, "🧹 Shot Gather after Demultiple")

## 🔧 Step 8: 노이즈 제거 (Denoise)

**이 셀을 실행하면:**
- 밴드패스 필터 (8-60 Hz)
- F-K 필터 (저속 성분 제거)
- Median 필터 (스파이크 제거)
- 노이즈 제거된 Shot Gather 시각화
- SNR 개선 계산

In [ ]:
print("🔧 노이즈 제거 중...")
print()
print("적용 기법:")
print("   1️⃣ 밴드패스 필터 (8-60 Hz)")
print("   2️⃣ F-K 필터 (저속 성분 제거, < 1500 m/s)")
print("   3️⃣ Median 필터 (스파이크 제거)")
print()

# 순차적 노이즈 제거
result = processor.denoise_bandpass_filter(demultipled, 8.0, 60.0)
result = processor.denoise_fk_filter(result, 1500)
denoised_shot = processor.denoise_median_filter(result, 5)

print("✅ 노이즈 제거 완료!")
print()

# 통계
residual = denoised_shot - with_direct  # 직접파 포함 원본과 비교
snr_after = 20 * np.log10(np.std(with_direct) / np.std(residual))

print("="*80)
print("📊 최종 결과")
print("="*80)
print(f"Denoised RMS:         {np.sqrt(np.mean(denoised_shot**2)):.6f}")
print(f"Residual RMS:         {np.sqrt(np.mean(residual**2)):.6f}")
print()
print(f"SNR (노이즈 추가 후):   {snr_before:.2f} dB")
print(f"SNR (처리 후):         {snr_after:.2f} dB")
print(f"SNR 개선:             {snr_after - snr_before:.2f} dB  ⬆️")
print("="*80)
print()

# 시각화
print("📈 노이즈 제거된 Shot Gather 시각화...")
processor.plot_shot_gather(denoised_shot, offsets, "🔧 Final Denoised Shot Gather")

## 📊 Step 9: 전체 비교 (4단계)

**이 셀을 실행하면:**
- Clean → Multiples → Noise → Final 4단계 비교
- 전체 처리 과정 시각적 확인

In [ ]:
print("📊 전체 비교 시각화 (4단계)...")
print()

titles = [
    'Clean (Direct Wave)',
    'With Multiples',
    'With Noise',
    'Final (Denoised)'
]

processor.plot_comparison_4(with_direct, with_multiples, noisy_shot, denoised_shot, offsets, titles)

print("✅ 비교 완료!")
print()
print("💡 처리 단계:")
print("   1️⃣ Clean: 반사파 + 직접파")
print("   2️⃣ Multiples: 해면 & 내부 multiple 추가")
print("   3️⃣ Noise: 해상 노이즈 추가 (선박, 스웰 등)")
print("   4️⃣ Final: Multiple 제거 + 노이즈 제거")

## 💾 Step 10: 데이터 저장 및 다운로드

**이 셀을 실행하면:**
- 4개 NPZ 파일 저장
- 자동 다운로드 (Colab)

In [ ]:
print("💾 데이터 저장 중...")
print()

# 데이터 저장
np.savez('shot_gather_clean.npz',
         shot_gather=with_direct,
         offsets=offsets,
         time=processor.time,
         model=model)
print("✅ shot_gather_clean.npz 저장 (직접파 포함)")

np.savez('shot_gather_with_multiples.npz',
         shot_gather=with_multiples,
         offsets=offsets,
         time=processor.time,
         model=model)
print("✅ shot_gather_with_multiples.npz 저장 (Multiple 추가)")

np.savez('shot_gather_noisy.npz',
         shot_gather=noisy_shot,
         offsets=offsets,
         time=processor.time,
         model=model)
print("✅ shot_gather_noisy.npz 저장 (노이즈 추가)")

np.savez('shot_gather_final.npz',
         shot_gather=denoised_shot,
         offsets=offsets,
         time=processor.time,
         model=model)
print("✅ shot_gather_final.npz 저장 (최종 처리)")
print()

# Colab에서 다운로드
try:
    from google.colab import files
    print("📥 다운로드 시작...")
    print()
    files.download('shot_gather_clean.npz')
    files.download('shot_gather_with_multiples.npz')
    files.download('shot_gather_noisy.npz')
    files.download('shot_gather_final.npz')
    print("✅ 모든 파일 다운로드 완료!")
except:
    print("ℹ️ 로컬 환경에서 실행 중 - 파일이 현재 디렉토리에 저장되었습니다.")

print()
print("="*80)
print("🎉 전체 워크플로우 완료!")
print("="*80)

## 🔬 Step 11: 추가 분석 - 단일 트레이스 비교

**선택사항: 특정 트레이스를 자세히 분석**

In [ ]:
# 중간 오프셋 트레이스 선택
trace_idx = len(offsets) // 2

print(f"🔬 트레이스 {trace_idx} 분석 (오프셋 = {offsets[trace_idx]:.0f} m)")
print()

# 플롯
fig, axes = plt.subplots(1, 4, figsize=(20, 6))

data_list = [with_direct, with_multiples, noisy_shot, denoised_shot]
titles = ['Clean', 'With Multiples', 'With Noise', 'Final']
colors = ['b', 'orange', 'r', 'g']

for ax, data, title, color in zip(axes, data_list, titles, colors):
    ax.plot(data[:, trace_idx], processor.time, color=color, linewidth=1.5)
    ax.set_xlabel('Amplitude', fontsize=11, fontweight='bold')
    ax.set_ylabel('Time (s)', fontsize=11, fontweight='bold')
    ax.set_title(f'{title}\n(Offset={offsets[trace_idx]:.0f}m)', fontsize=12, fontweight='bold')
    ax.invert_yaxis()
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"✅ 트레이스 {trace_idx} 비교 완료!")

## 📊 Step 12: 주파수 스펙트럼 분석

**선택사항: 주파수 도메인에서 비교**

In [ ]:
# 중간 트레이스 FFT
trace_idx = len(offsets) // 2
dt = processor.time[1] - processor.time[0]

print(f"📊 주파수 스펙트럼 분석 (트레이스 {trace_idx}, 오프셋 {offsets[trace_idx]:.0f}m)")
print()

# FFT 계산
freq = np.fft.rfftfreq(len(processor.time), dt)
clean_fft = np.abs(np.fft.rfft(with_direct[:, trace_idx]))
mult_fft = np.abs(np.fft.rfft(with_multiples[:, trace_idx]))
noisy_fft = np.abs(np.fft.rfft(noisy_shot[:, trace_idx]))
final_fft = np.abs(np.fft.rfft(denoised_shot[:, trace_idx]))

# 플롯
fig, ax = plt.subplots(figsize=(14, 6))

ax.plot(freq, clean_fft, 'b-', linewidth=2, label='Clean', alpha=0.8)
ax.plot(freq, mult_fft, color='orange', linewidth=2, label='With Multiples', alpha=0.7)
ax.plot(freq, noisy_fft, 'r-', linewidth=2, label='With Noise', alpha=0.6)
ax.plot(freq, final_fft, 'g-', linewidth=2, label='Final', alpha=0.8)

ax.set_xlabel('Frequency (Hz)', fontsize=13, fontweight='bold')
ax.set_ylabel('Amplitude Spectrum', fontsize=13, fontweight='bold')
ax.set_title(f'Frequency Spectrum Comparison (Offset={offsets[trace_idx]:.0f}m)', 
             fontsize=14, fontweight='bold')
ax.set_xlim([0, 100])
ax.legend(fontsize=12, loc='upper right')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("✅ 주파수 스펙트럼 분석 완료!")
print()
print("💡 관찰 포인트:")
print("   - Multiples: 반복 패턴으로 주파수 성분 증가")
print("   - Noise: 저주파 및 고주파 노이즈 증가")
print("   - Final: 필터링으로 원본에 가까워짐")

## 🎉 완료!

---

### 📝 요약

모든 단계를 실행하셨다면:

✅ 랜덤 지층 모델 생성 완료  
✅ Clean Shot Gather 생성 완료  
✅ 직접파 추가 완료 ⭐  
✅ Multiple 추가 완료 (해면 + 내부) ⭐  
✅ 해상 노이즈 추가 완료 (Ground Roll 제외) ⭐  
✅ Multiple 제거 완료 ⭐  
✅ 노이즈 제거 완료  
✅ 4개 파일 다운로드 완료

---

### 💾 생성된 파일

1. **shot_gather_clean.npz** - Clean (직접파 포함)
2. **shot_gather_with_multiples.npz** - Multiple 추가
3. **shot_gather_noisy.npz** - 해상 노이즈 추가
4. **shot_gather_final.npz** - 최종 처리

---

### 🌊 해상 탄성파 특징

- ✅ **직접파**: 해수층 통과 직접 도달
- ✅ **해면 Multiple**: 해수면 반사 (RC ≈ -0.95)
- ✅ **내부 Multiple**: 지층 경계면 다중 반사
- ✅ **해상 노이즈**: 선박, 스웰, 버스트
- ❌ **Ground Roll**: 해상에는 없음

---

### 🔄 다시 실행하기

새로운 랜덤 모델로 다시 실행하려면:
- **Step 2**부터 다시 실행하세요!
- 매번 다른 지층 모델과 결과를 얻을 수 있습니다.

---

### 📚 관련 링크

- **GitHub**: https://github.com/knocgp/seismic
- **상세 가이드**: [SHOT_GATHER_GUIDE.md](https://github.com/knocgp/seismic/blob/main/SHOT_GATHER_GUIDE.md)

---

**Made with ❤️ for Marine Seismic Data Processing**